In [ ]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.models import load_model
import numpy as np
import cv2

In [ ]:
facenet = cv2.dnn.readNet('models/deploy.prototxt', 'models/res10_300x300_ssd_iter_140000.caffemodel')
model = load_model('models/mask_detector.model')

In [ ]:
cap = cv2.VideoCapture('imgs/01.mp4')

while cap.isOpened():
    ret, img = cap.read()
    if not ret:
        break
    img = cv2.resize(img, (0,0), fx=0.3, fy=0.28)
    h, w = img.shape[:2]
    blob = cv2.dnn.blobFromImage(img, scalefactor=1., size = (300, 300), mean = (104., 177., 123.))
    facenet.setInput(blob)
    dets = facenet.forward()
    faces = []
    face_point = []
    for i in range (dets.shape[2]):
        confidence = dets[0, 0, i, 2]
        if confidence < 0.5:
            continue

        x1 = int(dets[0, 0, i, 3]*w)
        y1 = int(dets[0, 0, i, 4]*h)
        x2 = int(dets[0, 0, i, 5]*w)
        y2 = int(dets[0, 0, i ,6]*h)

        face = img[y1:y2, x1:x2]
        faces.append(face)
        face_point.append([x1, y1, x2, y2])
        
    for i, face in enumerate (faces):
        face_input = cv2.resize(face, dsize=(224,224))
        face_input = cv2.cvtColor(face_input, cv2.COLOR_BGR2RGB)
        face_input = preprocess_input(face_input)
        face_input = np.expand_dims(face_input, axis=0)

        mask, nomask = model.predict(face_input).squeeze()
        
        img = cv2.rectangle(img, pt1=(face_point[i][0], face_point[i][1]), pt2=(face_point[i][2], face_point[i][3]), color=(255,255,255), thickness=1, lineType=cv2.LINE_AA)
        if mask*100 < 90:
            text_color = (0, 0, 255)
            img = cv2.putText(img, 'no mask %.2f%%' % (mask * 100), (face_point[i][0], face_point[i][1]-5), cv2.FONT_HERSHEY_SIMPLEX, 0.3, text_color, thickness=1)
        else:
            text_color = (0, 255, 0)
            img = cv2.putText(img, 'mask %.2f%%' % (mask * 100), (face_point[i][0], face_point[i][1]-5), cv2.FONT_HERSHEY_SIMPLEX, 0.3, text_color, thickness=1)
        
    cv2.imshow('img', img)
    cv2.waitKey(1)
cv2.destroyAllWindows()